In [0]:
from pyspark.sql.functions import *
import pandas as pd

load = [(1,"A",100,"2025-08-01 08:30:55","9999-01-01 00:00:00",True),(2,"B",None,"2025-08-01 08:30:55","9999-01-01 00:00:00",True),(3,"C",400,"2025-08-01 08:30:55","9999-01-01 00:00:00",True),(4,"D",300,"2025-08-01 08:30:55","9999-01-01 00:00:00",True),(5,"E",None,"2025-08-01 08:30:55","9999-01-01 00:00:00",True),(6,"F",700,"2025-08-01 08:30:55","9999-01-01 00:00:00",True),(7,"G",1000,"2025-08-01 08:30:55","2025-08-05 08:30:55",False),(7,"G",600,"2025-08-05 08:30:55","9999-01-01 00:00:00",True)]

load_df = spark.createDataFrame(load,schema = "uid int, uname string, sal double, start_date string, end_date string, is_active boolean")
#load_df.show()

target_df = load_df.withColumn("start_date", to_timestamp("start_date"))\
    .withColumn("end_date",to_timestamp("end_date"))

target_df.show() 
target_df.dtypes 

In [0]:
target_hash_df = target_df.withColumn("hash",xxhash64("uid","uname","sal")).select("uid","uname","sal","hash")
target_hash_df.show()

In [0]:
new_users = [(4,"D",300),(5,"E",700),(6,"X",700),(8,"H",None),(9,"I",None)]
new_df = spark.createDataFrame(new_users,schema = "uid int , uname string,sal double")
new_df.show()

In [0]:
new_hash_df = new_df.withColumn("hash",xxhash64("uid","uname","sal")).select("uid","uname","sal","hash")
new_hash_df.show()

In [0]:
join_df = new_hash_df.join(target_hash_df,"hash","left_anti").drop("hash")
join_df.show()

In [0]:
save_df = join_df.withColumn("start_date", date_format(current_timestamp(),"yyyy-MM-dd HH:mm:ss"))\
    .withColumn("end_date",to_timestamp(lit("9999-01-01 00:00:00")))\
    .withColumn("is_active",lit(True))
    
save_df.show(truncate= False)

In [0]:
#target_df.show()
#save_df.show()

union_df = target_df.unionAll(save_df)
union_df.orderBy("uid").show()

In [0]:
from pyspark.sql.window import Window
windspec = Window.partitionBy("uid").orderBy(desc("start_date"))
rnk_df = union_df.withColumn("rnk", row_number().over(windspec))
rnk_df.show()


In [0]:
active_df = rnk_df.withColumn("is_active", when(col("rnk")==lit(2),False).otherwise(col("is_active")))
final_df= active_df.withColumn("end_date", when(col("rnk")==lit(2),lag("start_date").over(Window.partitionBy("uid").orderBy("rnk"))).otherwise(col("end_date")))
final_df.show()
    